## Tying it all together

In [10]:
import music21 as m21

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import copy
import cPickle

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.utils import np_utils
from keras.models import load_model

In [3]:
f = open('clean_bach.p','rb')
clean_bach = cPickle.load(f)
f.close()

clean_bach.head()

,BWV,orig_key,beats_per_measure,forward_note,back_note,forward_measure,back_measure,note_in_measure_position,quarter_in_measure_position,soprano_pitch_cat,alto_pitch_cat,tenor_pitch_cat,bass_pitch_cat,soprano_pitch_cont,alto_pitch_cont,tenor_pitch_cont,bass_pitch_cont
bwv421_0,bwv421,a minor,4,0,159,0,10,1,4,G4,D4,B-3,G2,1,-4,-8,-23
bwv421_1,bwv421,a minor,4,1,158,0,10,2,4,G4,D4,B-3,G2,1,-4,-8,-23
bwv421_2,bwv421,a minor,4,2,157,0,10,3,4,G4,D4,C4,A2,1,-4,-6,-21
bwv421_3,bwv421,a minor,4,3,156,0,10,4,4,G4,D4,C4,A2,1,-4,-6,-21
bwv421_4,bwv421,a minor,4,4,155,1,9,1,1,G4,D4,D4,B-2,1,-4,-4,-20


In [13]:
f = open('bach_dict.p','rb')
bach_dict = dill.load(f)
f.close()

bach_dict

RuntimeError: maximum recursion depth exceeded while calling a Python object

## Tying it all together

### Categorical Variables

#### target y variable - SOPRANO

In [37]:
# a series
# include BWV so that we can group by it later on
master_y_cat_sop = clean_bach[['BWV','soprano_pitch_cat']]
dummy_sop = pd.get_dummies(master_y_cat_sop['soprano_pitch_cat'])
    
dummy_sop.head(2)

,A4,A5,B-3,B-4,B-5,B3,B4,B5,C#4,C#5,...,E5,F#4,F#5,F4,F5,G#4,G#5,G4,G5,Rest
bwv421_0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
bwv421_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [45]:
[1 if x == sop_index else 0 for x in dummy_sop.columns]

Index([u'A4', u'A5', u'B-3', u'B-4', u'B-5', u'B3', u'B4', u'B5', u'C#4',
       u'C#5', u'C4', u'C5', u'C6', u'D#4', u'D#5', u'D4', u'D5', u'D6',
       u'E-4', u'E-5', u'E4', u'E5', u'F#4', u'F#5', u'F4', u'F5', u'G#4',
       u'G#5', u'G4', u'G5', u'Rest'],
      dtype='object')

#### target y variable - ALTO

In [38]:
# a series
master_y_cat_alto = clean_bach[['BWV','alto_pitch_cat']]
dummy_alto = pd.get_dummies(master_y_cat_alto['alto_pitch_cat'])

dummy_alto.head(2)

,A#4,A-3,A-4,A3,A4,B-3,B-4,B3,B4,C#4,...,F#5,F3,F4,F5,G#4,G-4,G3,G4,G5,Rest
bwv421_0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
bwv421_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### target y variable - TENOR

In [39]:
# a series
master_y_cat_tenor = clean_bach[['BWV','tenor_pitch_cat']]
dummy_tenor = pd.get_dummies(master_y_cat_tenor['tenor_pitch_cat'])
    
dummy_tenor.head(2)

,A#4,A-3,A-4,A3,A4,B-3,B-4,B3,B4,C#3,...,E4,F#3,F#4,F3,F4,G#3,G#4,G3,G4,Rest
bwv421_0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
bwv421_1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### target y variable - BASS

In [40]:
# a series
master_y_cat_bass = clean_bach[['BWV','bass_pitch_cat']]
dummy_bass = pd.get_dummies(master_y_cat_bass['bass_pitch_cat'])
    
dummy_bass.head(2)

,A#2,A#3,A-2,A-3,A2,A3,A4,B-1,B-2,B-3,...,F#4,F2,F3,F4,G#2,G#3,G2,G3,G4,Rest
bwv421_0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
bwv421_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


## Building the score

In [11]:
# Load the models

model_2 = load_model('../Model_2_Output/model_2.h5') # soprano
model_2a = load_model('../Model_2_Output/model_2a.h5') # alto
model_2b = load_model('../Model_2_Output/model_2b.h5') # tenor
model_2c = load_model('../Model_2_Output/model_2c.h5') # bass

In [161]:
# for every song in clean_bach, return the first four notes of the unique BWV
def create_model_2_scores(choice='argmax'):
    for i, (BWV, every_chorale) in enumerate(clean_bach.groupby('BWV')):
        # this should be in 'G4' format
        if i<3: # for troubleshooting
            initial_four_notes = every_chorale.ix[0,['soprano_pitch_cat', 'alto_pitch_cat', 
                                                 'tenor_pitch_cat', 'bass_pitch_cat']].values

            sop_note = initial_four_notes[0]
            alto_note = initial_four_notes[1]
            tenor_note = initial_four_notes[2]
            bass_note = initial_four_notes[3]

            #################################################################################
            # for that song, return the same four notes as dummies and return that TOGETHER as 
            # an array prev_four_notes for the model AND as the first row of FOUR separate
            # arrays - one for each part

            # convert each one to the equivalent dummy encoding to enter into the LSTM model
            soprano_dummy = [1 if x == sop_note else 0 for x in dummy_sop.columns]
            alto_dummy = [1 if x == alto_note else 0 for x in dummy_alto.columns]
            tenor_dummy = [1 if x == tenor_note else 0 for x in dummy_tenor.columns]
            bass_dummy = [1 if x == bass_note else 0 for x in dummy_bass.columns]

            # create the initial four notes to set up the LSTM
            prev_four_notes = list(soprano_dummy)
            prev_four_notes.extend(alto_dummy)
            prev_four_notes.extend(tenor_dummy)
            prev_four_notes.extend(bass_dummy)

            prev_four_notes = np.array([prev_four_notes])

            # create the array of dummy variables per part; we will use arg max or random choice
            soprano_composed_dummies = np.array([soprano_dummy])
            alto_composed_dummies = np.array([alto_dummy])
            tenor_composed_dummies = np.array([tenor_dummy])
            bass_composed_dummies = np.array([bass_dummy])

            print i, '\n'#, 'initial_four_notes', initial_four_notes, type(initial_four_notes)
    #         print 'prev_four_notes', prev_four_notes

            #################################################################################
            # now, for that song, have it predict 255 notes to make up "a chorale". 
            # 255 = the average length of a bach chorale, in 16th notes
            # 
            
            for i in range(15): ############# USE 255

                # shape prev_four_notes into your X variable
                n_dummy_vars = prev_four_notes.shape[1]

                prev_four_notes_RS = np.reshape(prev_four_notes, (1, 
                                           1, n_dummy_vars))

                # predict sop using the LSTM model
                new_sop_dummy = model_2.predict_proba(prev_four_notes_RS)

                # predict alto
                new_alto_dummy = model_2a.predict_proba(prev_four_notes_RS)

                # predict tenor
                new_tenor_dummy = model_2b.predict_proba(prev_four_notes_RS)

                # predict bass
                new_bass_dummy = model_2c.predict_proba(prev_four_notes_RS)

    #             print 'new_sop_dummy', len(new_sop_dummy[0]), new_sop_dummy
    #             print 'new_bass_dummy', len(new_alto_dummy), new_alto_dummy


                # add each prediction to its respective array (sop, alto, tenor, bass)
                # this will store all the notes that we predicted
                soprano_composed_dummies = np.concatenate([soprano_composed_dummies, new_sop_dummy], 
                                                          axis=0)
                alto_composed_dummies = np.concatenate([alto_composed_dummies, new_alto_dummy], 
                                                          axis=0)
                tenor_composed_dummies = np.concatenate([tenor_composed_dummies, new_tenor_dummy], 
                                                          axis=0)
                bass_composed_dummies = np.concatenate([bass_composed_dummies, new_bass_dummy], 
                                                          axis=0)

                # set prev_four_notes = the concat of the four arrays
                # we will use this to predict the next note in the for loop
                prev_four_notes = np.concatenate([new_sop_dummy, new_alto_dummy, new_tenor_dummy,
                                                 new_bass_dummy], axis=1)

#             print 'soprano_composed_dummies', np.sum(soprano_composed_dummies, axis=1),'\n'

            #################################################################################
            # now, for that song, have it identify which note INDEX was actually chosen
            # use arg max or random_choice, as indicated by the user
            # if user chose argmax, then use the note with the highest probability. this runs the risk
            # of producing 1) a song that sounds the same as the original or 2) something that 
            # mistakenly sounds the same.
            # for that reason, we are including random choice as an option
            
            if choice=='argmax':
                soprano_composed_indices = np.argmax(soprano_composed_dummies, axis=1)
                alto_composed_indices = np.argmax(alto_composed_dummies, axis=1)
                tenor_composed_indices = np.argmax(tenor_composed_dummies, axis=1)
                bass_composed_indices = np.argmax(bass_composed_dummies, axis=1)

#                 print 'soprano_composed_indices', soprano_composed_indices.shape, soprano_composed_indices, '\n'
            
            # if the user chose random choice, pick a value using the probabilities generated
            # by the model
            elif choice=='random_choice':
                # soprano - set up empty np array
                soprano_composed_indices = np.array([])
                
                for item in soprano_composed_dummies:
                    # need to divide item by sum of item because the elements do not sum to 1
                    # probably some rounding error from the model (numbers are v small)
                    random_choice = np.random.choice(range(item.shape[0]), size=1, p=item/np.sum(item))
                    soprano_composed_indices = np.append(soprano_composed_indices, random_choice)
                   
                # alto
                alto_composed_indices = np.array([])
                
                for item in alto_composed_dummies:
                    random_choice = np.random.choice(range(item.shape[0]), size=1, p=item/np.sum(item))
                    alto_composed_indices = np.append(alto_composed_indices, random_choice)
                
                # tenor
                tenor_composed_indices = np.array([])
                
                for item in tenor_composed_dummies:
                    random_choice = np.random.choice(range(item.shape[0]), size=1, p=item/np.sum(item))
                    tenor_composed_indices = np.append(tenor_composed_indices, random_choice)
                
                # bass
                bass_composed_indices = np.array([])
                
                for item in bass_composed_dummies:
                    random_choice = np.random.choice(range(item.shape[0]), size=1, p=item/np.sum(item))
#                     print 'random_choice', random_choice
                    bass_composed_indices = np.append(bass_composed_indices, random_choice)
#             print 'bass_composed_dummies', bass_composed_dummies        
            #################################################################################
            # first create four new arrays, using the index and applying it to the columns
            # names to identify the pitch it represents.
            #
            # then create four new arrays, each one representing the m21.note.Note objects
            # from music21.
            #
            # take those arrays and put together a score


            # soprano
            the_columns = dummy_sop.columns
    
            # use the index from index_best_recommendation to identify the pitch it 
            # matches from the_columns
            return_pitch_array = np.vectorize(lambda x: the_columns[x])
            soprano_composed_pitches = return_pitch_array(soprano_composed_indices)
            
            create_m21_part = np.vectorize(return_m21_object)
            soprano_m21_part = create_m21_part(soprano_composed_pitches)

            # alto
            the_columns = dummy_alto.columns
    
            # use the index from index_best_recommendation to identify the pitch it 
            # matches from the_columns
            return_pitch_array = np.vectorize(lambda x: the_columns[x])
            alto_composed_pitches = return_pitch_array(alto_composed_indices)
            
            create_m21_part = np.vectorize(return_m21_object)
            alto_m21_part = create_m21_part(alto_composed_pitches)

            # tenor
            the_columns = dummy_tenor.columns
    
            # use the index from index_best_recommendation to identify the pitch it 
            # matches from the_columns
            return_pitch_array = np.vectorize(lambda x: the_columns[x])
            tenor_composed_pitches = return_pitch_array(tenor_composed_indices)

            create_m21_part = np.vectorize(return_m21_object)
            tenor_m21_part = create_m21_part(tenor_composed_pitches)

            # bass
            the_columns = dummy_bass.columns
    
            # use the index from index_best_recommendation to identify the pitch it 
            # matches from the_columns
            return_pitch_array = np.vectorize(lambda x: the_columns[x])
            bass_composed_pitches = return_pitch_array(bass_composed_indices)
            
            create_m21_part = np.vectorize(return_m21_object)
            bass_m21_part = create_m21_part(bass_composed_pitches)

#             print soprano_m21_part
#             print alto_m21_part
#             print tenor_m21_part
#             print bass_m21_part
            
            #################################################################################
            # make a m21 score         
            
            ###### soprano
            soprano = m21.stream.Part()

            sop_instr = m21.instrument.Instrument()
            sop_instr.partName = 'Soprano'
            soprano.append(sop_instr)

            soprano.append(m21.clef.TrebleClef())
            soprano.append(m21.key.Key('G'))
            soprano.append(m21.meter.TimeSignature('4/4'))

            for thisThing in soprano_m21_part:
                soprano.append(thisThing)

            soprano_barline = m21.bar.Barline()
            soprano_barline.style = 'final'
            soprano.append(soprano_barline)
            
            soprano.makeMeasures(inPlace=True)

            ###### alto
            alto = m21.stream.Part()

            alto_instr = m21.instrument.Instrument()
            alto_instr.partName = 'Alto'
            alto.append(alto_instr)

            alto.append(m21.clef.TrebleClef())
            alto.append(m21.key.Key('G'))
            alto.append(m21.meter.TimeSignature('4/4'))

            for thisThing in alto_m21_part:
                alto.append(thisThing)

            alto_barline = m21.bar.Barline()
            alto_barline.style = 'final'
            alto.append(alto_barline)
            
            alto.makeMeasures(inPlace=True)

            ###### tenor

            tenor = m21.stream.Part()

            tenor_instr = m21.instrument.Instrument()
            tenor_instr.partName = 'Tenor'
            tenor.append(tenor_instr)

            tenor.append(m21.clef.BassClef())
            tenor.append(m21.key.Key('G'))
            tenor.append(m21.meter.TimeSignature('4/4'))

            for thisThing in tenor_m21_part:
                tenor.append(thisThing)

            tenor_barline = m21.bar.Barline()
            tenor_barline.style = 'final'
            tenor.append(tenor_barline)
            
            tenor.makeMeasures(inPlace=True)


            ###### bass
            bass = m21.stream.Part()

            bass_instr = m21.instrument.Instrument()
            bass_instr.partName = 'Bass'
            bass.append(tenor_instr)

            bass.append(m21.clef.BassClef())
            bass.append(m21.key.Key('G'))
            bass.append(m21.meter.TimeSignature('4/4'))

            for thisThing in bass_m21_part:
                bass.append(thisThing)

            bass_barline = m21.bar.Barline()
            bass_barline.style = 'final'
            bass.append(bass_barline)
            
            bass.makeMeasures(inPlace=True)


            ##
            the_score = m21.stream.Stream()
            the_score.insert(0, m21.metadata.Metadata())
            title = BWV + '_CLBach'
            the_score.metadata.title = title
            the_score.metadata.composer = 'C.L. Bach'
            the_score.insert(0, soprano)
            the_score.insert(0, alto)
            the_score.insert(0, tenor)
            the_score.insert(0, bass)
            staff_layout = m21.layout.StaffGroup([soprano, alto, tenor, bass], symbol='brace')
            staff_layout.barTogether = 'Mensurstrich'            
            the_score.append(staff_layout)
           
            
    the_score.show('midi')
            
# for every part:
    # use arg max or random choice on the vector of probabilities
#     np.random.choice(range(len(notes)), size=1, p=prediction)[0]
    # identify what pitch that is by tying that to a column and 
    # create four new arrays, one for each part, of m21.note.Notes

create_model_2_scores(choice='random_choice')

0 

1/1 [==============================] - 0s
1 

1/1 [==============================] - 0s
2 

1/1 [==============================] - 0s


midi.base.py: WARNING: Conversion error for <MidiEvent PROGRAM_CHANGE, t=0, track=1, channel=1>: Got incorrect data for <MidiEvent PROGRAM_CHANGE, t=0, track=1, channel=1> in .data: None,cannot parse Program Change; ignored.
midi.base.py: WARNING: Conversion error for <MidiEvent PROGRAM_CHANGE, t=0, track=2, channel=1>: Got incorrect data for <MidiEvent PROGRAM_CHANGE, t=0, track=2, channel=1> in .data: None,cannot parse Program Change; ignored.
midi.base.py: WARNING: Conversion error for <MidiEvent PROGRAM_CHANGE, t=0, track=3, channel=1>: Got incorrect data for <MidiEvent PROGRAM_CHANGE, t=0, track=3, channel=1> in .data: None,cannot parse Program Change; ignored.
midi.base.py: WARNING: Conversion error for <MidiEvent PROGRAM_CHANGE, t=0, track=4, channel=1>: Got incorrect data for <MidiEvent PROGRAM_CHANGE, t=0, track=4, channel=1> in .data: None,cannot parse Program Change; ignored.


In [111]:
def return_m21_object(pitch_name):
    if pitch_name == 'Rest':
        a = m21.note.Rest()

    else:
        a = m21.note.Note(pitch_name)
    
    a.quarterLength = 0.25
    
    return a

In [85]:
an_array = np.array([[1.00832254e-01,   1.96539736e-06,   1.01624323e-07 ,  3.06925480e-03,
    1.96289378e-07  , 2.45973695e-07,   8.01680237e-03,   1.42437079e-06,
    2.55937493e-06  , 1.02665572e-05,   2.38430562e-06,   2.38309172e-03,
    6.55275812e-07  , 4.23878555e-06,   1.22554397e-07,   6.65651809e-04,
    8.88249185e-03  , 4.11631754e-06,   3.77493816e-05,   1.96859673e-05,
    2.31566941e-04  , 8.90845404e-05,   1.22133561e-03,   1.04716280e-04,
    4.31974069e-04  , 1.66276586e-05,   3.90458918e-05,   5.95390929e-06,
    8.71955335e-01  , 3.25968518e-04,   1.64313964e-03],
                    [ 8.25216472e-02,   3.18995058e-06 ,  1.51822775e-07  , 2.23245728e-03,
    2.66729558e-07 ,  2.72873422e-07 ,  7.29342503e-03 ,  2.74481067e-06,
    3.10380074e-06 ,  1.82785843e-05 ,  3.12068300e-06 ,  2.24744133e-03,
    9.03224702e-07 ,  4.57606984e-06 ,  1.51574810e-07 ,  7.24998128e-04,
    1.26035493e-02 ,  4.73167029e-06 ,  3.87536711e-05 ,  3.04142450e-05,
    2.93164980e-04 ,  1.11462352e-04 ,  1.70249445e-03 ,  1.36044211e-04,
    4.48202016e-04 ,  1.90536903e-05 ,  4.15682953e-05 ,  7.54522489e-06,
    8.86989117e-01 ,  3.96308838e-04 ,  2.12085666e-03]])

In [131]:
soprano_composed_pitches = np.array([])
np.append(soprano_composed_pitches,'G4')
np.append(soprano_composed_pitches,'G4')

array(['G4'], 
      dtype='|S32')

In [95]:
# random_choice = np.vectorize(lambda x: np.random.choice(range(x.shape[0]), size=1, p=x))
# random_choice(list(an_array))
# np.random.choice(range(x.shape[0]), size=1, p=x))
for item in an_array:
    print np.random.choice(range(item.shape[0]), size=1, p=item)

[28]
[28]


In [18]:
np.array(first_starting_notes['soprano_pitch_cat'])

# np.array(first_starting_notes)

array(['B4', 'G4', 'C5', 'B-4', 'B-4', 'B-4', 'G4', 'G4', 'B-4', 'D5',
       'G4', 'D4', 'D5', 'G4', 'D5', 'D5', 'G4', 'D4', 'D4', 'G4', 'E4',
       'D5', 'D5', 'D5', 'D5', 'C4', 'G4', 'G4', 'D5', 'C5', 'D5', 'G4',
       'D5', 'D5', 'G4', 'G4', 'G4', 'D5', 'E5', 'G4', 'G4', 'G4', 'G4',
       'D5', 'D4', 'G4', 'G5', 'B-4', 'G5', 'D5', 'D4', 'G4', 'G4', 'G4',
       'G4', 'G4', 'A4', 'A4', 'D5', 'G4', 'D5', 'D5', 'D5', 'G4', 'D5',
       'G4', 'B4', 'G4', 'D5', 'B4', 'D5', 'G4', 'G5', 'B4', 'G4', 'G4',
       'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'D4', 'G4', 'Rest', 'B-4', 'D4',
       'B-4', 'G4', 'G4', 'G4', 'D5', 'G4', 'G4', 'G4', 'G4', 'G4', 'D5',
       'D5', 'D5', 'B4', 'G4', 'D5', 'D5', 'B4', 'B4', 'B4', 'B4', 'G4',
       'G4', 'F4', 'G4', 'D5', 'G4', 'B4', 'D5', 'G4', 'G4', 'B4', 'G4',
       'G4', 'G4', 'G4', 'G4', 'B4', 'D5', 'D5', 'G4', 'F4', 'B-5', 'G4',
       'D5', 'G4', 'G4', 'G5', 'G5', 'G4', 'B4', 'B4', 'B4', 'B4', 'D5',
       'G4', 'D5', 'B4', 'D5', 'G4', 'G4', 'G4'

# Create a function that transforms your model results back into music21 data. 

In [966]:
# use this as the base template to put the new predictions in?
new_CL_bach = clean_bach[['BWV', 'soprano_pitch_cat', 'alto_pitch_cat', 'tenor_pitch_cat',
       'bass_pitch_cat', 'soprano_pitch_cont', 'alto_pitch_cont',
       'tenor_pitch_cont', 'bass_pitch_cont']]

## CATEGORICAL

In [1060]:
LSTM_predict_to_music21_score(test_predictions, 0,32580)

Did you predict soprano in this model (Y/N)?Y
Did you predict alto in this model (Y/N)?N
Did you predict tenor in this model (Y/N)?N
Did you predict bass in this model (Y/N)?N
the_array:
13820 [[  3.93820410e-05   4.62009439e-06   3.52374554e-05 ...,   1.64113529e-02
    3.72923000e-06   2.65301478e-05]
 [  3.93820410e-05   4.62009439e-06   3.52374554e-05 ...,   1.64113529e-02
    3.72923000e-06   2.65301478e-05]
 [  3.93820410e-05   4.62009439e-06   3.52374554e-05 ...,   1.64113529e-02
    3.72923000e-06   2.65301478e-05]
 ..., 
 [  1.04425965e-06   2.21450555e-05   1.88121510e-06 ...,   7.13190390e-03
    1.87975718e-04   5.80103406e-05]
 [  1.04425965e-06   2.21450555e-05   1.88121510e-06 ...,   7.13190390e-03
    1.87975718e-04   5.80103406e-05]
 [  1.04425965e-06   2.21450555e-05   1.88121510e-06 ...,   7.13190390e-03
    1.87975718e-04   5.80103406e-05]]
pitch_name_array:
13820 ['F#4' 'F#4' 'F#4' ..., 'B3' 'B3' 'B3']
length of an_array 13820
length of pitch_name_array: 13820
leng

ValueError: Length of values does not match length of index

In [1062]:
len(test_predictions[0])

42

In [1059]:
new_CL_bach.head()

,BWV,soprano_pitch_cat,alto_pitch_cat,tenor_pitch_cat,bass_pitch_cat,soprano_pitch_cont,alto_pitch_cont,tenor_pitch_cont,bass_pitch_cont,bass_composed
bwv421_0,bwv421,G4,D4,B-3,G2,1,-4,-8,-23,<numpy.lib.function_base.vectorize object at 0...
bwv421_1,bwv421,G4,D4,B-3,G2,1,-4,-8,-23,<numpy.lib.function_base.vectorize object at 0...
bwv421_2,bwv421,G4,D4,C4,A2,1,-4,-6,-21,<numpy.lib.function_base.vectorize object at 0...
bwv421_3,bwv421,G4,D4,C4,A2,1,-4,-6,-21,<numpy.lib.function_base.vectorize object at 0...
bwv421_4,bwv421,G4,D4,D4,B-2,1,-4,-4,-20,<numpy.lib.function_base.vectorize object at 0...


In [1042]:
# ok yeah i'm proud of this code, woot
# assign this function to a new dataframe, then apply the create_score function to the DF
# need lag so we know whether to fill in where the data has been offset and therefore missing

def LSTM_predict_to_music21_score(proba_matrix, lag, test_start_index):

    # which part is this for? do a series of raw inputs to get it to set up parts
    predicted_parts = []
    
    # 4 different raw inputs to figure out which parts. seek user input 
    # do a for loop to go through all teh possible parts. each time the user says yes, we did
    # predict that voice part, add taht part to the predictedparts list'''
    for voice_part in ['soprano', 'alto', 'tenor', 'bass']:
        use = raw_input('Did you predict {0} in this model (Y/N)?'.format(voice_part))
        if use == 'Y':
            predicted_parts.append(voice_part)
        
        
    # identify the number of categorical variables per voice part so that the for loop
    # can set aside the right number of columns
    the_dict = {'soprano' : n_y_cat_variables_sop,
               'alto' : n_y_cat_variables_alto,
               'tenor' : n_y_cat_variables_tenor,
               'bass' : n_y_cat_variables_bass}

    # go thorugh each predicted part and set aside those values for that part
    for i, each_part in enumerate(predicted_parts): # could be ['soprano', 'alto', 'tenor', 'bass']
        # if it's the first one, then the beginning index needs to be 0.
        # if it's not the first one, then you'll have set begin_index from the 
        # -previous- iteration.
        if i == 0:
            begin_index = 0

        end_index = the_dict[each_part]

        the_list = []

            # for each row in proba_matrix, let's split out the values that belong to this part
        for row in proba_matrix:
            row = row[begin_index:]
            row = row[:end_index]
            the_list.append(row)

        # after doing this iteration, set the stage for the next iteration by incrementing 
        # begin_index by how much you've advanced in this iteration. ie. if soprano is first 
        # and len(soprano) is 31, then this iteration will look like 0:31. the NEXT iteration
        # will look like [31:31+the alto length]
        begin_index = end_index

        # turn this list into an array
        the_array = np.array(the_list)
        print 'the_array:\n',len(the_array), the_array
        
        # turn this into a series of notes
        return turn_proba_array_into_a_series_of_notes(the_array, each_part, lag, test_start_index)
        
        # take teh series of notes, and BY SONG, put it back into a stream/score
        # you can take all the notes and put them sequentially in a list
        # then call get_measures or something like that, and music21 will set up measure markings
        # be sure to set up other needed stuff like key and time signatures.
        # put EACH SCORE INTO A DICT LIKE BACH_DICT
#         create_score(a_df)

In [1015]:
def turn_proba_array_into_a_series_of_notes(an_array, each_part, lag=0, test_start_index=0):
    # determine the model's best recommendation from probabiltiies of the predicted note
    index_best_recommendation = an_array.argmax(axis=1)
    
    # make this into a list for ease
    index_best_recommendation = index_best_recommendation.tolist()
    
    # get a list of all the column names for that part
    columns_by_part_dict = {
        'soprano' : master_y_cat_dummy_sop.columns,
        'alto' : master_y_cat_dummy_alto.columns,
        'tenor' : master_y_cat_dummy_tenor.columns,
        'bass' : master_y_cat_dummy_bass.columns
    }
   
    # exclude the one column BWV since that is not a pitch and will not be exported by the model
    the_columns = columns_by_part_dict[each_part][1:]
    
    # use the index from index_best_recommendation to identify the pitch it 
    # matches from the_columns
    return_pitch_array = np.vectorize(lambda x: the_columns[x])
    pitch_name_array = return_pitch_array(index_best_recommendation)
    
    print 'pitch_name_array:\n', len(pitch_name_array), pitch_name_array
    # use the DF new_CL_bach. split it up into distinct groupbys by BWV. 
    # add a column with the notes from an_array.
    # if lag was used, add the 'true' notes from new_CL_bach
    
    # create a new list to hold all the notes in the composed version
    composed_list = []
    
    print 'length of an_array', len(an_array)
    print 'length of pitch_name_array:', len(pitch_name_array)
    print 'length of new_CL_bach:', len(new_CL_bach)
    
    # do this by bwv groupby since lag is at the SONG level
    for bwv_groupby in new_CL_bach.groupby(['BWV']):
        # each iterative groupby item is a tuple with the BWV as the key and the rows
        # as the value. so I need to access the rows via bwv_groupby[1]
        
        # define a variable for the number of rows in the groupby
        n_groupby_rows = len(bwv_groupby[1])
        
        # identify the column of "true" notes from teh new_CL_bach df
        desired_column_from_new_CL_bach = each_part + '_pitch_cat'
        
        # if we used a train/test split, then we need to start filling in with the 
        # actual data, since we never predicted any of those notes
        # ALSO GOING TO ASSUME THAT WE ARE NOT EVER GOING TO DO TRAIN TEST SPLIT AND LAG 
        # AT THE SAME TIME        
        while test_start_index > 0:
            
            # if hte number of rows is less than test_start_index, then fill composed_list
            # completely with notes from new_CL_bach. # notes = n_groupby_rows
            if n_groupby_rows <= test_start_index:
                composed_list.extend(bwv_groupby[1][desired_column_from_new_CL_bach][0:n_groupby_rows]) 
                

                # then reduce test_start_index by the number of entries we used
                test_start_index -= n_groupby_rows
    
            # if instead, the number of rows is GREATER than test_start_index, then we need
            # to fill composed_list partly with notes from new_CL_bach and partly from an_array
            elif n_groupby_rows > test_start_index:
                # fill with # notes = test_start_index
                composed_list.extend(bwv_groupby[1][desired_column_from_new_CL_bach][:test_start_index]) 
                # fill the rest of the notes from an_array (# = n_groupby_rows - test_start_index)
                n_array_used = n_groupby_rows - test_start_index
                composed_list.extend(pitch_name_array[:n_array_used])
                
                # then set test_start_index=0 because we used it up
                test_start_index = 0 
    
                # and since we also used up notes from an_array, set it equal to whatever is left
                an_array = an_array[n_array_used:]
                                     
        # if we didn't use a train test split, then perhaps we used a lag. or maybe not. 
        # here is the code we can use to test it
        else:
            # this is how many we'll take from the array - first grab from new_CL_bach to
            # fill for the lag. then the remainder is from an_array
            n_array_used = n_groupby_rows - lag
            
            # fill from the true notes for n=lag
            composed_list.extend(bwv_groupby[1][desired_column_from_new_CL_bach][:lag]) # choose notes from the appropriate part
            
            # for the remainder of the notes, fill from an_array
            composed_list.extend(pitch_name_array[0:n_array_used])
            
            # and since we also used up notes from an_array, set it equal to whatever is left
            pitch_name_array = pitch_name_array[n_array_used:]
    
    print 'composed_list: \n', len(composed_list), composed_list[0:50], composed_list[-50:]
    
    # turn pitch_name_array into m21.note.Note objects with the correct duration
    return_m21_object_array = np.vectorize(return_m21_object)
    m21_array = return_m21_object_array(np.array(composed_list))
    
    print 'm21_array:\n', len(m21_array), m21_array[0:50], m21_array[-50:]
    
    # create a new column in the DF and put those pitches in - dealing iwth th elag issue
    new_column = each_part + '_composed'
    new_CL_bach[new_column] = m21_array

In [1083]:
a_list = [1,2,3,4,5]
# turn pitch_name_array into m21.note.Note objects with the correct duration
return_m21_object_array = np.vectorize(return_m21_object)
m21_array = return_m21_object_array(np.array(a_list))

print 'm21_array:\n', len(m21_array)
m21_array

# # create a new column in the DF and put those pitches in - dealing iwth th elag issue
# new_column = each_part + '_composed'
# new_CL_bach[new_column] = m21_array

m21_array:
5


array([<music21.note.Note C#>, <music21.note.Note D>,
       <music21.note.Note E->, <music21.note.Note E>, <music21.note.Note F>], dtype=object)

# LSTM Model

### Model 1 - predict S given three A, T, and B variables
- variables:

    - soprano

    - alto
    
    - tenor
    
    - bass
    
    - no other ones
    
- categorical or continuous:

    - categorical

- train test split

    - 30%
    
- variables as features or time lag? time lag as features or time lag?

    - variables as features; time lag as time lag
    
- normalize/standardize?

    - no
    
- time lagged: 

    - NO
    
- input nodes:

    - no
    
- intermediate nodes:

    - 10 LSTM
    
- activation:

    - softmax
    
- optimizer:

    - adam
    
- number epochs:

    - 10
    
- number batch size

    - 20

- dropout

    - NA
    
- CNN

    - NA


- split out train test set 30%
- define data set - X and y variables
- reshape X

In [1067]:
master_y_cat_dummy_sop.columns

Index([u'BWV', u'A4', u'A5', u'B-3', u'B-4', u'B-5', u'B3', u'B4', u'B5',
       u'C#4', u'C#5', u'C4', u'C5', u'C6', u'D#4', u'D#5', u'D4', u'D5',
       u'D6', u'E-4', u'E-5', u'E4', u'E5', u'F#4', u'F#5', u'F4', u'F5',
       u'G#4', u'G#5', u'G4', u'G5', u'Rest'],
      dtype='object')

In [1068]:
# create y var for model 1; make it an array
columns = [x for x in master_y_cat_dummy_sop.columns if x != 'BWV']
y1 = master_y_cat_dummy_sop[columns].values
y1.shape

(46676, 31)

In [1069]:
y1[0] 

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  1.,  0.,  0.])

In [1070]:
# create X vars for model 1; make it an array
columns_not_needed = [x for x in master_X_cat_dummies.columns if x[:7]=='soprano']

columns_not_needed.extend(['BWV','beats_per_measure', 'forward_note',
                     'back_note', 'forward_measure', 'back_measure',
                     'note_in_measure_position',
                      'quarter_in_measure_position'])


columns_needed = [x for x in master_X_cat_dummies.columns if x not in columns_not_needed]


X1 = master_X_cat_dummies[columns_needed].values
X1.shape

(46676, 110)

In [1071]:
# train test split
# 70% of the number of rows
train_end_index = int(round(y1.shape[0] * 0.7))

# 70% of the rows is 32673. This is in the middle of a piece. I'm going to round down
# to the nearest song break to make things cleaner.

train_end_index = 32580
y1_train, y1_test = y1[:train_end_index], y1[train_end_index:]
X1_train, X1_test = X1[:train_end_index], X1[train_end_index:]

In [1072]:
X1_train[0]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  1.,  0.,  0.,  0.])

In [1073]:
y1_train.shape, y1_test.shape, X1_train.shape, X1_test.shape

((32580, 31), (14096, 31), (32580, 110), (14096, 110))

In [1074]:
# set variables
time_lag = 1 
# actually it is 0 since we are not doing time series, but for the 
# purpose of the 3D X matrix we will make it 1
n_features = X1.shape[1]

In [1075]:
# reshape X to be [samples, time steps, features]
X1_train_RS = np.reshape(X1_train, (X1_train.shape[0], 
                                       time_lag, 
                                      n_features))

X1_test_RS = np.reshape(X1_test, (X1_test.shape[0], 
                                       time_lag, 
                                      n_features))

In [1076]:
X1_train_RS[0:2]

array([[[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  1.,  0.,  0.,  0.]],

       [[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  

In [1077]:
# SET UP THE MODEL 
model_1 = Sequential()
model_1.add(LSTM(10, input_dim=110))
model_1.add(Dense(n_y_cat_variables_sop, activation='softmax'))
model_1.compile(loss='categorical_crossentropy', optimizer='adam',
             metrics=['accuracy'])

In [1078]:
print model_1.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lstm_25 (LSTM)                   (None, 10)            4840        lstm_input_25[0][0]              
____________________________________________________________________________________________________
dense_25 (Dense)                 (None, 31)            341         lstm_25[0][0]                    
Total params: 5181
____________________________________________________________________________________________________
None


In [1079]:
model_1.fit(X1_train_RS, y1_train, nb_epoch=10, batch_size=20, verbose=2)

Epoch 1/10
26s - loss: 2.2123 - acc: 0.3574
Epoch 2/10
26s - loss: 1.4979 - acc: 0.5322
Epoch 3/10
22s - loss: 1.3344 - acc: 0.5797
Epoch 4/10
23s - loss: 1.2616 - acc: 0.5957
Epoch 5/10
26s - loss: 1.2184 - acc: 0.6056
Epoch 6/10
31s - loss: 1.1882 - acc: 0.6131
Epoch 7/10
26s - loss: 1.1656 - acc: 0.6161
Epoch 8/10
24s - loss: 1.1477 - acc: 0.6222
Epoch 9/10
23s - loss: 1.1329 - acc: 0.6266
Epoch 10/10
22s - loss: 1.1202 - acc: 0.6294


In [1080]:
train_predictions_prob_1 = model_1.predict_proba(X1_train_RS)

32480/32580 [============================>.] - ETA: 0s

In [1081]:
test_predictions_1 = model_1.predict(X1_test_RS)

In [1082]:
test_predictions_1[0]

array([  8.24480355e-01,   1.53144756e-05,   8.69268391e-08,
         1.05491569e-02,   3.19459559e-06,   2.89442823e-06,
         3.26918215e-02,   2.43674640e-06,   8.19634897e-06,
         4.81363822e-05,   5.44216618e-06,   3.52562144e-02,
         4.53779057e-06,   6.68009579e-06,   7.46130198e-08,
         1.61147909e-04,   7.55081698e-02,   1.05484160e-05,
         2.06186305e-05,   1.98399153e-04,   1.10477369e-04,
         7.96304201e-04,   1.91268360e-03,   8.42815265e-04,
         3.24643857e-04,   1.08208996e-03,   5.44857212e-05,
         4.74974831e-06,   1.57751646e-02,   1.22324855e-04,
         7.61253091e-07], dtype=float32)

In [1054]:
# result is [loss, accuracy]
model_1.evaluate(X1_test_RS, y1_test, batch_size=20)

14000/14096 [============================>.] - ETA: 0s

[1.2173820120154559, 0.61776390759470989]

let's try that again but playing around with batch_size

In [1036]:
# SET UP THE MODEL 
model_1a = Sequential()
model_1a.add(LSTM(10, input_dim=110))
model_1a.add(Dense(n_y_cat_variables_sop, activation='softmax'))
model_1a.compile(loss='categorical_crossentropy', optimizer='adam',
             metrics=['accuracy'])

In [1037]:
model_1a.fit(X1_train_RS, y1_train, nb_epoch=10, batch_size=100, verbose=2)

Epoch 1/10
6s - loss: 2.9143 - acc: 0.2601
Epoch 2/10
5s - loss: 2.0788 - acc: 0.3766
Epoch 3/10
5s - loss: 1.7343 - acc: 0.4747
Epoch 4/10
5s - loss: 1.5378 - acc: 0.5329
Epoch 5/10
5s - loss: 1.4253 - acc: 0.5594
Epoch 6/10
5s - loss: 1.3555 - acc: 0.5762
Epoch 7/10
5s - loss: 1.3084 - acc: 0.5880
Epoch 8/10
4s - loss: 1.2745 - acc: 0.5960
Epoch 9/10
4s - loss: 1.2485 - acc: 0.6033
Epoch 10/10
5s - loss: 1.2286 - acc: 0.6078


In [675]:
# chart to show the probability of each note
###################### NEED TO DO ######################

what happens if we do stateful?

In [1038]:
# SET UP THE MODEL 
model_1b = Sequential()
model_1b.add(LSTM(10, batch_input_shape=(1, 1, 110), stateful=True))
model_1b.add(Dense(n_y_cat_variables_sop, activation='softmax'))
model_1b.compile(loss='categorical_crossentropy', optimizer='adam',
             metrics=['accuracy'])

In [1039]:
print model_1b.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lstm_23 (LSTM)                   (1, 10)               4840        lstm_input_23[0][0]              
____________________________________________________________________________________________________
dense_23 (Dense)                 (1, 31)               341         lstm_23[0][0]                    
Total params: 5181
____________________________________________________________________________________________________
None


In [ ]:
model_1b.fit(X1_train_RS, y1_train, nb_epoch=10, batch_size=1, verbose=2)

In [ ]:
train_predictions_prob_1b = model_1b.predict_proba(X1_train_RS, batch_size=1)

In [ ]:
test_predictions_1b = model_1b.predict(X1_test_RS, batch_size=1)

In [ ]:
test_predictions_1b[0]

In [ ]:
# result is [loss, accuracy]
model_1b.evaluate(X1_test_RS, y1_test, batch_size=1)

do this again, turning off the 'shuffling' of the training data set

In [ ]:
# SET UP THE MODEL 
model_1c = Sequential()
model_1c.add(LSTM(10, batch_input_shape=(4, 1, 110), stateful=True))
model_1c.add(Dense(n_y_cat_variables_sop, activation='softmax'))
model_1c.compile(loss='categorical_crossentropy', optimizer='adam',
             metrics=['accuracy'])

In [ ]:
print model_1c.summary()

In [ ]:
model_1c.fit(X1_train_RS, y1_train, nb_epoch=10, batch_size=4, verbose=2, shuffle=False)

In [ ]:
train_predictions_prob_1c = model_1c.predict_proba(X1_train_RS, batch_size=4)

In [ ]:
test_predictions_1c = model_1c.predict(X1_test_RS, batch_size=4)

In [ ]:
test_predictions_1c[0]

In [ ]:
# result is [loss, accuracy]
model_1c.evaluate(X1_test_RS, y1_test, batch_size=4)

In [903]:
'''need to see what this loos like!!!!!!!'''

'need to see what this loos like!!!!!!!'

### Let's take a look at what our baseline is going to be. for every combination of the three parts alto, tenor, and bass, what are the possible soprano options and how likely are they relative to each other?


In [ ]:
# each value is the alto-tenor-bass combined into one variable
three_parts = clean_bach[['alto_pitch_cat','tenor_pitch_cat',
           'bass_pitch_cat']].apply(lambda x: x[0]+x[1]+x[2], axis=1)

three_parts.head(2)

In [ ]:
# combine the sop and other parts together into one df
# there are 46676 notes in total

combos = pd.concat([clean_bach[['soprano_pitch_cat']],three_parts], axis=1)
combos.columns = ['soprano_pitch_cat','other_parts']
print combos.shape
combos.head(2)

In [ ]:
# if we group by the 46676 notes by the two columns, we can count the number of times that 
# sop/other-parts combo comes up.

# the goal is to come up wtih some number that describes the baseline - actually something
# a little differnet than baseline - describe how often am i actually arleady giving the 
# machine the answer since tehre is only ever one sop-ATB combo for that ATB?
# 

n_sop_parts_combos = combos.groupby(['other_parts', 'soprano_pitch_cat']).agg(len).reset_index()
n_sop_parts_combos.rename(columns={0:'this_SATB_combo_count'}, inplace=True)
print n_sop_parts_combos.shape
n_sop_parts_combos.head()

In [ ]:
# this is the number of distinct SATB combos there are for every ATB.
# ie. if value=1, then that ATB is only matched with ONE sop.
# ie. if value=3, then that ATN is matched with THREE sopranos

other_parts_count = n_sop_parts_combos['other_parts'].value_counts()
print other_parts_count.sum()
other_parts_count = pd.DataFrame(other_parts_count)
print other_parts_count.head()
print other_parts_count.tail()

In [ ]:
# other parts count EXCLUDING THOSE ATBs THAT ARE ONLY MATCHED WITH *ONE* SOP

# this means that 2807 out of the 3862 S-ATB combos, or 73%, are NOT obvious answers.

print other_parts_count[other_parts_count>1].sum()
other_parts_count[other_parts_count>1].head()

### this says there are 3862 possible SATB combos, and of them, 2807, or 73% are not 100% obvious what soprano to choose. this means that 27% of the time, the computer knows EXACTLY what the "right" answer is.

### what about of the 46,676 notes we had?

In [ ]:
# ok well let's look at how many of the notes that I fed into the model were
# based on ATB bases that were easy to predict (ie they only had one option)

themerged = pd.merge(n_sop_parts_combos, other_parts_count, how='inner', left_on='other_parts',
        right_index=True)

In [ ]:
themerged.drop('other_parts_x', inplace=True, axis=1)
themerged.rename(columns={'other_parts_y' : 'n_distinct_SATB_for_this_ATB'}, inplace=True)


In [ ]:
themerged.head()

In [ ]:
themerged.shape

In [ ]:
themerged[themerged['n_distinct_SATB_for_this_ATB']>1].head()
# pooh = themerged[themerged['n_distinct_SATB_for_this_ATB']>1][0]
# print pooh.head()
# np.sum(pooh)
np.sum(themerged[themerged['n_distinct_SATB_for_this_ATB']>1]['n_distinct_SATB_for_this_ATB'])

### Alright. 11,361. This is the number of notes where there is more than one soprano note for the given three lower parts. So, 24% of the 46,676 notes are NOT a toss-up. which means the computer is pretty much guaranteed to be right 76% of the time. And the remaining 24% of the time, it is likely that there are a few most-likely options, which would increase the baseline accuracy up from an automatic 76%.

What if we ran a quick correlation matrix to see how well the notes are correlated?

In [ ]:
# S vs ATB
n_sop_parts_combos.columns


In [ ]:
# y1 is the soprano part
# X1 is the ATB parts altogether

y1X1 = pd.concat([pd.DataFrame(y1), pd.DataFrame(X1)], axis=1)

## Now I am going to run the LSTM model to predict the NEXT note given all previous FOUR notes.
### Models 2-5 will predict soprano, alto, tenor and bass based on the previous stack of notes.

-------------
### i need to create a function to provide me with time-lagged y OR X *by song*.  this will solve the problem that using data in row(t-1) to predict row(t) is NOT HELPFUL AT THE END OF BEGINNING OF SONGS. because the end of a song DOES NOT predict the beginning of the next one.

#### this should work for y AND X. 
-----------------
## HOWEVER

### i will have to do this every time i need a new time lag. in other words, if i want to do time lag of 2 periods, i will have to do:
- create_timelagged_DF_BY_SONG(df, 1)

- create_timelagged_DF_BY_SONG(df, 2)

--------------------

In [862]:
def create_timelagged_DF_BY_SONG(df, X_or_y, lag):
    if X_or_y == 'y':
        return_df = df.groupby('BWV').apply(lambda x: x[lag:])
    if X_or_y == 'X':
        return_df = df.groupby('BWV').apply(lambda x: x[:-lag])
    return_df.drop('BWV',axis=1, inplace=True)        
    return return_df

## Model 2 (was Model K for Kiefer =))
### Predict soprano (time t), given predictor variables SATB (time t-1)

- variables:

    - soprano

    - alto
    
    - tenor
    
    - bass
    
    - no other ones
    
- categorical or continuous:

    - categorical

- train test split

    - ** 0%
    
- variables as features or time lag? time lag as features or time lag?

    - variables as features; time lag as time lag
    
- normalize/standardize?

    - no
    
- time lagged: 

    - ** Yes - 1
    
- input nodes:

    - no
    
- intermediate nodes:

    - 10 LSTM
    
- activation:

    - softmax
    
- optimizer:

    - adam
    
- number epochs:

    - 10
    
- number batch size

    - 20

- dropout

    - NA
    
- CNN

    - NA


In [ ]:
master_y_cat_dummy_sop.head()

In [ ]:
# create y var for model 1; make it an array
y2 = create_timelagged_DF_BY_SONG(master_y_cat_dummy_sop, 'y', 1).values
y2.shape

In [ ]:
y2

In [ ]:
# create X vars for model 1; make it an array
columns_not_needed = ['beats_per_measure', 'forward_note',
                     'back_note', 'forward_measure', 'back_measure',
                     'note_in_measure_position',
                      'quarter_in_measure_position']

columns_needed = [x for x in master_X_cat_dummies.columns if x not in columns_not_needed]

intermediate = master_X_cat_dummies[columns_needed]

X2 = create_timelagged_DF_BY_SONG(intermediate, 'X',1).values

X2.shape

In [ ]:
# do NOT train test split. we are going to just train and then write new music.
y2_train, y2_test = y2, y2
X2_train, X2_test = X2, X2

In [ ]:
X2_train[0]

In [ ]:
y2.shape, y2_test.shape, X2_train.shape, X2_test.shape

In [ ]:
# set variables
time_lag = 1 
# actually it is 0 since we are not doing time series, but for the 
# purpose of the 3D X matrix we will make it 1
n_features = X2.shape[1]

In [ ]:
# reshape X to be [samples, time steps, features]
X2_train_RS = np.reshape(X2_train, (X2_train.shape[0], 
                                       time_lag, 
                                      n_features))

X2_test_RS = np.reshape(X2_test, (X2_test.shape[0], 
                                       time_lag, 
                                      n_features))

In [ ]:
X2_train_RS[0:2]

In [ ]:
# SET UP THE MODEL 
model_2 = Sequential()
model_2.add(LSTM(10, input_dim=141))
model_2.add(Dense(n_y_cat_variables_sop, activation='softmax'))
model_2.compile(loss='categorical_crossentropy', optimizer='adam',
             metrics=['accuracy'])

In [ ]:
print model_2.summary()

In [ ]:
model_2.fit(X2_train_RS, y2_train, nb_epoch=20, batch_size=20, verbose=2)

In [ ]:
train_predictions_prob_2 = model.predict_proba(X2_train_RS)

In [ ]:
train_predictions_prob_2[0:1]

In [ ]:
test_predictions_2 = model_2.predict(X2_test_RS)

In [ ]:
test_predictions_2[0]

In [ ]:
# result is [loss, accuracy]
model_2.evaluate(X2_test_RS, y2_test, batch_size=20)

In [ ]:
model_2.save('../Model_2_Output/model_2.h5')

In [ ]:
model_2 = load_model('model_2.h5')

In [ ]:
''' i have just run this model - need to do the same for alto, tenor and bass. 
Then create a function or something to predict things all together based on 
predictiosn just made. and then put it all toether into one big dataframe'''

In [ ]:
'''need to see what this loos like!!!!!!!'''

### Model 2a:  Predict alto (time t), given predictor variables SATB (time t-1)

In [ ]:
master_y_cat_dummy_alto.head()

In [ ]:
# create y var for model 1; make it an array
y2a = create_timelagged_DF_BY_SONG(master_y_cat_dummy_alto, 'y', 1).values
y2a.shape

In [ ]:
y2a

In [ ]:
# create X vars for model 1; make it an array
columns_not_needed = ['beats_per_measure', 'forward_note',
                     'back_note', 'forward_measure', 'back_measure',
                     'note_in_measure_position',
                      'quarter_in_measure_position']

columns_needed = [x for x in master_X_cat_dummies.columns if x not in columns_not_needed]

intermediate = master_X_cat_dummies[columns_needed]

X2a = create_timelagged_DF_BY_SONG(intermediate, 'X',1).values

X2a.shape

In [ ]:
# as before, do NOT train test split this since we will predict from scratch

# train_end_index = int(round(y1.shape[0] * 0.7))

y2a_train, y2a_test = y2a, y2a
X2a_train, X2a_test = X2a, X2a

In [ ]:
X2a_train[0]

In [ ]:
y2a_train.shape, y2a_test.shape, X2a_train.shape, X2a_test.shape

In [ ]:
# set variables
time_lag = 1 
# actually it is 0 since we are not doing time series, but for the 
# purpose of the 3D X matrix we will make it 1
n_features = X2a.shape[1]

In [ ]:
# reshape X to be [samples, time steps, features]
X2a_train_RS = np.reshape(X2a_train, (X2a_train.shape[0], 
                                       time_lag, 
                                      n_features))

X2a_test_RS = np.reshape(X2a_test, (X2a_test.shape[0], 
                                       time_lag, 
                                      n_features))

In [ ]:
X2a_train_RS[0:2]

In [ ]:
# SET UP THE MODEL 
model_2a = Sequential()
model_2a.add(LSTM(10, input_dim=141))
model_2a.add(Dense(n_y_cat_variables_alto, activation='softmax'))
model_2a.compile(loss='categorical_crossentropy', optimizer='adam',
             metrics=['accuracy'])

In [ ]:
print model_2a.summary()

In [ ]:
model_2a.fit(X2a_train_RS, y2a_train, nb_epoch=20, batch_size=20, verbose=2)

In [ ]:
train_predictions_prob_2a = model_2a.predict_proba(X2a_train_RS)

In [ ]:
test_predictions_2a = model_2a.predict(X2a_test_RS)

In [ ]:
# result is [loss, accuracy]
model_2a.evaluate(X2a_test_RS, y2a_test, batch_size=20)

In [ ]:
test_predictions_2a[0]

In [ ]:
model_2a.save('./Model_2_Output/model_2a.h5')

In [ ]:
model_2a = load_model('model_2a.h5')

### Model 2b:  Predict tenor (time t), given predictor variables SATB (time t-1)

In [ ]:
master_y_cat_dummy_tenor.head()

In [ ]:
# create y var for model 1; make it an array
y2b = create_timelagged_DF_BY_SONG(master_y_cat_dummy_tenor, 'y', 1).values
y2b.shape

In [ ]:
y2b

In [ ]:
# create X vars for model 1; make it an array
columns_not_needed = ['beats_per_measure', 'forward_note',
                     'back_note', 'forward_measure', 'back_measure',
                     'note_in_measure_position',
                      'quarter_in_measure_position']

columns_needed = [x for x in master_X_cat_dummies.columns if x not in columns_not_needed]

intermediate = master_X_cat_dummies[columns_needed]

X2b = create_timelagged_DF_BY_SONG(intermediate, 'X',1).values

X2b.shape

In [ ]:
# no train test split
# train_end_index = int(round(y1.shape[0] * 0.7))

y2b_train, y2b_test = y2b, y2b
X2b_train, X2b_test = X2b, X2b

In [ ]:
X2b_train[0]

In [ ]:
y2b_train.shape, y2b_test.shape, X2b_train.shape, X2b_test.shape

In [ ]:
# set variables
time_lag = 1 
# actually it is 0 since we are not doing time series, but for the 
# purpose of the 3D X matrix we will make it 1
n_features = X2b.shape[1]

In [ ]:
# reshape X to be [samples, time steps, features]
X2b_train_RS = np.reshape(X2b_train, (X2b_train.shape[0], 
                                       time_lag, 
                                      n_features))

X2b_test_RS = np.reshape(X2b_test, (X2b_test.shape[0], 
                                       time_lag, 
                                      n_features))

In [ ]:
X2b_train_RS[0:2]

In [ ]:
# SET UP THE MODEL 
model_2b = Sequential()
model_2b.add(LSTM(10, input_dim=141))
model_2b.add(Dense(n_y_cat_variables_tenor, activation='softmax'))
model_2b.compile(loss='categorical_crossentropy', optimizer='adam',
             metrics=['accuracy'])

In [ ]:
print model_2b.summary()

In [ ]:
model_2b.fit(X2b_train_RS, y2b_train, nb_epoch=20, batch_size=20, verbose=2)

In [ ]:
train_predictions_prob_2b = model_2b.predict_proba(X2b_train_RS)

In [ ]:
train_predictions_prob_2b[0:1]

In [ ]:
test_predictions_2b = model_2b.predict(X2b_test_RS)

In [ ]:
test_predictions_2b[0]

In [ ]:
# result is [loss, accuracy]
model_2b.evaluate(X2b_test_RS, y2b_test, batch_size=20)

In [ ]:
model_2b.save('./Model_2_Output/model_2b.h5')

In [ ]:
model_2b = load_model('model_2b.h5')

### Model 2c:  Predict bass (time t), given predictor variables SATB (time t-1)

In [ ]:
master_y_cat_dummy_bass.head()

In [ ]:
# create y var for model 1; make it an array
y2c = create_timelagged_DF_BY_SONG(master_y_cat_dummy_bass, 'y', 1).values
y2c.shape

In [ ]:
y2c

In [ ]:
# create X vars for model 1; make it an array
columns_not_needed = ['beats_per_measure', 'forward_note',
                     'back_note', 'forward_measure', 'back_measure',
                     'note_in_measure_position',
                      'quarter_in_measure_position']

columns_needed = [x for x in master_X_cat_dummies.columns if x not in columns_not_needed]

intermediate = master_X_cat_dummies[columns_needed]

X2c = create_timelagged_DF_BY_SONG(intermediate, 'X',1).values

X2c.shape

In [ ]:
# NO train test split
# train_end_index = int(round(y1.shape[0] * 0.7))

y2c_train, y2c_test = y2c, y2c
X2c_train, X2c_test = X2c, X2c

In [ ]:
y2c.shape

In [ ]:
X2c_train[0]

In [ ]:
y2c_train.shape, y2c_test.shape, X2c_train.shape, X2c_test.shape

In [ ]:
# set variables
time_lag = 1 
# actually it is 0 since we are not doing time series, but for the 
# purpose of the 3D X matrix we will make it 1
n_features = X2c.shape[1]

In [ ]:
# reshape X to be [samples, time steps, features]
X2c_train_RS = np.reshape(X2c_train, (X2c_train.shape[0], 
                                       time_lag, 
                                      n_features))

X2c_test_RS = np.reshape(X2c_test, (X2c_test.shape[0], 
                                       time_lag, 
                                      n_features))

In [ ]:
X2c_train_RS[0:2]

In [ ]:
# SET UP THE MODEL 
model_2c = Sequential()
model_2c.add(LSTM(10, input_dim=141))
model_2c.add(Dense(n_y_cat_variables_bass, activation='softmax'))
model_2c.compile(loss='categorical_crossentropy', optimizer='adam',
             metrics=['accuracy'])

In [ ]:
print model_2c.summary()

In [ ]:
model_2c.fit(X2c_train_RS, y2c_train, nb_epoch=20, batch_size=20, verbose=2)

In [ ]:
train_predictions_prob_2c = model_2c.predict_proba(X2c_train_RS)

In [ ]:
train_predictions_prob_2c[0:1]

In [ ]:
test_predictions_2c = model_2c.predict(X2c_test_RS)

In [ ]:
# result is [loss, accuracy]
model_2c.evaluate(X2c_test_RS, y2c_test, batch_size=20)

In [ ]:
model_2c.save('./Model_2_Output/model_2c.h5')

In [ ]:
model_2c = load_model('model_2c.h5')

kiefer's suggestion - to be applied to a later model

In [ ]:
##################### thanks kiefer but i want to save this for a later model #####################

# create X vars for model 1; make it an array
columns_not_needed = ['beats_per_measure', 'forward_note',
                     'back_note', 'forward_measure', 'back_measure',
                     'note_in_measure_position',
                      'quarter_in_measure_position']

#columns_needed = [x for x in master_X_cat_dummies.columns if x not in columns_not_needed]
X1 = master_X_cat_dummies.values[0:-1]

# For these variables do min max scaling by song:
# - forward_note, back_note
# - note_in_measure_position
# - quarter_in_measure_position
# - forward_measure, back measure
#
# Essentially just min max scale per song everything.
# 
# Leave out beats_per_measure: it is irrelevant after min max scaling by song.
# no this is not true because beat 3 in 4 beat measure has meaning
# DO use min max scaler on beats per measure
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
X1 = mms.fit_transform(X1)

X1.shape

### Using MCMC

### K Nearest Neighbors Model

# IGMORE EVERYTHING PAST THIS. THIS *WAS MODEL 2* BUT HAS BEEN REMOVED

- variables:

    - soprano

    - alto
    
    - tenor
    
    - bass
    
    - no other ones
    
- categorical or continuous:

    - categorical

- train test split

    - 30%
    
- variables as features or time lag? time lag as features or time lag?

    - variables as features; time lag as time lag
    
- normalize/standardize?

    - no - the data is still just 0s and 1s
    
- time lagged: 

    - YES time lag 1 note
    
- input nodes:

    - no
    
- intermediate nodes:

    - 10 LSTM
    
- activation:

    - softmax
    
- optimizer:

    - adam
    
- number epochs:

    - 10
    
- number batch size

    - 20

- dropout

    - NA
    
- CNN

    - NA


- split out train test set 30%
- define data set - X and y variables
- reshape X

In [ ]:
# create y var for model 1; make it an array
y2 = master_y_cat_dummy.values

In [ ]:
# create X vars for model 1; make it an array
columns_not_needed = ['beats_per_measure', 'forward_note',
                     'back_note', 'forward_measure', 'back_measure',
                     'note_in_measure_position',
                      'quarter_in_measure_position']

columns_needed = [x for x in master_X_cat_dummies.columns if x not in columns_not_needed]
X2 = master_X_cat_dummies[columns_needed].values
X2.shape

## I think the next section is worthless becuase i needed to do this BY SONG.

In [ ]:
testy = np.array([1,2,3,4,5, 6])
testX = np.array(
    [[6, 12, 18],
     [7, 13, 19],    
     [8, 14, 20],
     [9, 15, 21],
     [10, 16, 22],
     [11, 17, 23]])

In [777]:
testy_a, testy_b = create_XY(testy, lag=2)

In [778]:
testy_a, testy_b

(array([[1, 2],
        [2, 3],
        [3, 4],
        [4, 5]]), array([3, 4, 5, 6]))

In [ ]:
testX_a, testX_b = create_XY(testX, lag=2)

In [ ]:
testX_a, testX_b

In [ ]:
# target y variable is testy_b
testy_b

In [ ]:
# x variables are:
# testX_a, testX_b, and testy_a
# print testX_a.shape, testX_b.shape, testy_a.shape
# time-aged X | real-time X | time-lagged y
# 3 = number of features
# 4 # of samples
# 2 = # of lags

# first run shapes are
# (3, 4, 2) (4, 3) (4, 2)

# we need it to be:
# (# samples, # time lags, # features(parts))
# (4, 2, 3)

# print 'testX_a by itself\n',testX_a, testX_a.shape
# print 'testX_a, reshaped \n',testX_a.reshape((4,2,3))
# print 'testX_a swapped axes \n',testX_a.swapaxes(0,1).swapaxes(1,2)

timelagged_X = testX_a.swapaxes(0,1).swapaxes(1,2)
realtime_X = testX_b.reshape((4,1,3))
timelagged_y = testy_a.reshape((4,2,1))

In [ ]:
realtime_X.shape

In [ ]:
# np.append(timelagged_X, np.append(realtime_X, timelagged_y, axis=1), axis=1)

# appending on axis 1 because realtime_X is another time lag
almost_there = np.append(timelagged_X, realtime_X, axis=1)
almost_there.shape
# np.append(almost_there, timelagged_y, axis=2)

In [ ]:
a = np.arange(18).reshape(9,2)
b = a.reshape(3,3,2)#.swapaxes(0,2)

In [ ]:
a

In [ ]:
b

In [ ]:
b.swapaxes(0,2)

In [776]:
# from A, create new_A and lagged_B based on time lag requested 
# if A = [1,2,3,4,5,6] and timelag = 1, then
# new_A = [2,3,4,5,6] and lagged_B = [1,2,3,4,5]
# (modified from kiefer lab)

# kiefer solution AMENDED
# works for 1D and 2D input
def create_XY(timeseries, lag=1):
    ts = timeseries#.ravel()
    y = ts[lag:]
    X = []# make this an array instead 
    for i in range(lag):
        current_pred = ts[i:]
        current_pred = current_pred[:len(y)]
        X.append(current_pred)
        
    if len(ts.shape) == 1:
        return_X = np.array(X).T
        
    else:
        return_X = np.array(X).T
    return return_X, y 

# def create_lagged_AB_from_A(timeseries_A, lag=1):
#     ts = timeseries_A#.ravel()
#     print "you entered: \n",type(ts), ts
#     new_A = ts[lag:]
#     print 'new_A:\n', type(new_A), new_A
#     lagged_B = []
#     for i in range(lag):
#         current_pred = ts[i:]
#         current_pred = current_pred[:len(new_A)]
#         lagged_B.append(current_pred)
#         print 'lagged_B:\n', type(lagged_B), lagged_B
    
#     if len(ts.shape) == 1:
#         return_B = np.array(lagged_B).T
    
#     else:
#         return_B = np.empty(new_A.shape,dtype=int)
#         print 'return_B: \n', return_B
        
#         for i, timelag_item in enumerate(np.array(lagged_B)):
#             print 'timelag item: ', i, '\n',timelag_item
#             return_B = np.append(return_B, timelag_item, axis=1)
            
# #             return_B = np.array(lagged_B)[0], np.array(lagged_B)[1]

#     print 'return_B: \n', return_B
#     return new_A, return_B

In [ ]:
a = np.array([[[2, 8],
 [3, 7]],
[[4, 6],
 [5, 5]],
[ [6, 4],
 [7, 3]]])

b = np.array([[4, 6],
 [5, 5]])

c = np.array(
[ [6, 4],
 [7, 3]])

In [ ]:
print a.shape, b.shape, c.shape

In [ ]:
np.array([b,c])

## back to our normal programming - NOPE PROBABLY GOING TO IGNORE THIS TOO

In [ ]:
# train test split
# 70% of the number of rows
train_end_index = int(round(y2.shape[0] * 0.7))

y2_train, y2_test = y2[:train_end_index], y2[train_end_index:]
X2_train, X2_test = X2[:train_end_index], X2[train_end_index:]

In [ ]:
X2_train[0:10]

In [ ]:
y1_train.shape, y1_test.shape, X1_train.shape, X1_test.shape

In [ ]:
# set variables
time_lag = 1 
# actually it is 0 since we are not doing time series, but for the 
# purpose of the 3D X matrix we will make it 1
n_features = X1.shape[1]

In [ ]:
# reshape X to be [samples, time steps, features]
X1_train_RS = np.reshape(X1_train, (X1_train.shape[0], 
                                       time_lag, 
                                      n_features))

X1_test_RS = np.reshape(X1_test, (X1_test.shape[0], 
                                       time_lag, 
                                      n_features))

In [ ]:
X1_train_RS[0:2]

In [ ]:
model = Sequential()
model.add(LSTM(10, input_dim=141))
model.add(Dense(n_y_cat_variables, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',
             metrics=['accuracy'])

In [ ]:
print model.summary()

In [ ]:
model.fit(X1_train_RS, y1_train, nb_epoch=10, batch_size=20, verbose=2)

In [ ]:
train_predictions_prob = model.predict_proba(X1_train_RS)

In [ ]:
train_predictions_prob[0:1]

In [ ]:
test_predictions = model.predict(X1_test_RS)

In [ ]:
test_predictions[0]

In [ ]:
model.evaluate(X1_test_RS, y1_test, batch_size=20)

In [ ]:
# chart to show the probability of each note

## Creating target and predictor variables
#### Create a master y and X for each categorical and continuous
### KEEP THESE AS DATAFRAMES

#### isolating predictor X variables from the DF. 

In [ ]:
clean_bach.columns

In [ ]:
master_X_cat = clean_bach[[u'BWV',u'beats_per_measure', u'forward_note',
       u'back_note', u'forward_measure', u'back_measure',
       u'note_in_measure_position', u'quarter_in_measure_position',
       u'soprano_pitch_cat', u'alto_pitch_cat', u'tenor_pitch_cat',
       u'bass_pitch_cat']]

#### turn categorical predictor X variables into DUMMY VARIABLES

In [ ]:
parts_df = master_X_cat[['soprano_pitch_cat', 'alto_pitch_cat', 
                         u'tenor_pitch_cat', u'bass_pitch_cat']]

In [ ]:
parts_df_dummy = pd.get_dummies(parts_df)

In [ ]:
parts_df_dummy.head(2)

### combining cat DUMMY variables with other features

In [ ]:
master_X_cat_dummies = pd.concat(
    [master_X_cat[[u'BWV',u'beats_per_measure', u'forward_note',u'back_note', 
                  u'forward_measure', u'back_measure',
                  u'note_in_measure_position', 
                  u'quarter_in_measure_position']],
    parts_df_dummy],
    axis=1)

In [ ]:
master_X_cat_dummies.head()

In [ ]:
list(master_X_cat_dummies.columns)

### Continuous Version

In [ ]:
# a series
master_y_cont = clean_bach['soprano_pitch_cont']

In [ ]:
master_X_cont = clean_bach[[u'beats_per_measure', u'forward_note',
       u'back_note', u'forward_measure', u'back_measure',
       u'note_in_measure_position', u'quarter_in_measure_position',
       u'soprano_pitch_cont', u'alto_pitch_cont', u'tenor_pitch_cont',
       u'bass_pitch_cont']]